<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания №19


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Subscription в C#, который будет представлять подписки на
различные услуги. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте полиморфизм с перекрытием и прегегрузкой методов, а также generic классы

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [1]:
using System;
using System.Collections.Generic;
using System.Linq; // добавлено для использования в generic классе

// интерфейс для подписок, к которым можно применить промокод
public interface IPromoCodeApplicable
{
    // метод для применения промокода
    void ApplyPromoCode(string promoCode);
}

// интерфейс для сервисов, имеющих профиль пользователя
public interface IUserProfile
{
    // свойство для имени пользователя
    string UserName { get; set; }
    // метод для обновления профиля
    void UpdateProfile(string newName);
}


// базовый класс для подписок
public class Subscription
{
    // через get set реализуем инкапсуляцию
    public int SubscriptionId { get; set; }
    // т.к. данные не выставлены всем "напоказ", а контролируются через эти аксессоры
    public string ServiceName { get; set; }
    // можно будет ещё вписать внутрь логику, какие данные будут приниматься, если это понадобится
    public double Cost { get; set; }
    
    public DateTime StartDate { get; private set; } // дата начала подписки
    public bool IsActive { get; private set; }      // статус активности

    // конструктор для инициализации объекта
    public Subscription(int id, string serviceName, double cost)
    {
        SubscriptionId = id;
        ServiceName = serviceName;
        Cost = cost;
        StartDate = DateTime.Now; // устанавливаем текущую дату при создании
        IsActive = true;          // по умолчанию подписка активна
    }

    // метод для взаимодействия с другой подпиской
    public void ApplyDiscountFrom(Subscription otherSubscription, double discountPercentage)
    {
        // пример взаимодействия: одна подписка предоставляет скидку на другую
        double discountAmount = otherSubscription.Cost * (discountPercentage / 100);
        this.Cost -= discountAmount;
        Console.WriteLine($"Применена скидка {discountPercentage}% от подписки '{otherSubscription.ServiceName}'. Новая стоимость '{this.ServiceName}': {this.Cost:C}");
    }
    
    public void CancelSubscription()
    {
        IsActive = false;
        Console.WriteLine($"Подписка на '{ServiceName}' была отменена");
    }
    
    public int GetSubscriptionAgeInDays()
    {
        return (int)(DateTime.Now - StartDate).TotalDays;
    }

    public virtual double CalculateMonthlyCost()
    {
        return Cost;
    }
    
    // шаг 1: реализация полиморфизма через перегрузку и перекрытие

    // метод для перекрытия (overriding) в дочерних классах
    public virtual void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес.");
    }
    
    // перегрузка (overloading) метода extendsubscription: версия без параметров
    public void ExtendSubscription()
    {
        // продление на 1 месяц по умолчанию
        ExtendSubscription(1); 
    }
    
    // перегрузка (overloading) метода extendsubscription: версия с бонусной скидкой
    public void ExtendSubscription(int months, double bonusDiscount)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. с дополнительной скидкой {bonusDiscount}%!");
    }


    public virtual string GetSubscriptionDetails()
    {
        return $"ID: {SubscriptionId}, Услуга: {ServiceName}, Базовая стоимость: {Cost:C}, Активна: {IsActive}";
    }
}

// класс наследуется от subscription и реализует два интерфейса
public class OnlineServiceSubscription : Subscription, IUserProfile, IPromoCodeApplicable
{
    public int MaxUsers { get; set; }
    public double UsedStorageGB { get; set; } 
    public string UserName { get; set; } 

    public OnlineServiceSubscription(int id, string serviceName, double costPerUser, int maxUsers, string userName)
        : base(id, serviceName, costPerUser)
    {
        MaxUsers = maxUsers;
        UserName = userName;
        UsedStorageGB = 0; 
    }
    
    public void CheckStorageUsage()
    {
        Console.WriteLine($"Использование хранилища: {UsedStorageGB} GB");
    }

    public void UpdateProfile(string newName)
    {
        Console.WriteLine($"Имя пользователя для '{ServiceName}' изменено с '{UserName}' на '{newName}'");
        UserName = newName;
    }

    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "CLOUD20")
        {
            Cost *= 0.8; 
            Console.WriteLine($"Промокод 'CLOUD20' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    public override double CalculateMonthlyCost()
    {
        return Cost * MaxUsers;
    }
}

// класс наследуется от subscription и реализует один интерфейс
public class StreamingSubscription : Subscription, IPromoCodeApplicable
{
    public int MaxStreams { get; set; }
    public bool HasOfflineMode { get; set; } 

    public StreamingSubscription(int id, string serviceName, double cost, int maxStreams, bool hasOfflineMode)
        : base(id, serviceName, cost)
    {
        MaxStreams = maxStreams;
        HasOfflineMode = hasOfflineMode;
    }
    
    public void ToggleOfflineMode()
    {
        HasOfflineMode = !HasOfflineMode;
        Console.WriteLine($"Офлайн-режим для '{ServiceName}' теперь {(HasOfflineMode ? "включен" : "выключен")}");
    }
    
    public void ApplyPromoCode(string promoCode)
    {
        if (promoCode == "MUSIC15")
        {
            Cost *= 0.85; 
            Console.WriteLine($"Промокод 'MUSIC15' применен! Новая базовая стоимость: {Cost:C}");
        }
        else
        {
            Console.WriteLine("Неверный промокод");
        }
    }

    // пример перекрытия (overriding) метода из базового класса
    public override void ExtendSubscription(int months)
    {
        Console.WriteLine($"Подписка на '{ServiceName}' продлена на {months} мес. Специальное предложение: +1 месяц в подарок!");
    }
}

public class VideoSubscription : Subscription
{
    public string VideoQuality { get; set; }
    public List<string> IncludedChannels { get; set; }

    public VideoSubscription(int id, string serviceName, double cost, string videoQuality)
        : base(id, serviceName, cost)
    {
        VideoQuality = videoQuality;
        IncludedChannels = new List<string>(); 
    }
    
    public void DisplayChannels()
    {
        Console.WriteLine("Включенные каналы:");
        foreach (var channel in IncludedChannels)
        {
            Console.WriteLine($"- {channel}");
        }
    }

    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Качество видео: {VideoQuality}";
    }
}

public class PremiumVideoSubscription : VideoSubscription
{
    public bool HasPersonalManager { get; set; } 

    public PremiumVideoSubscription(int id, string serviceName, double cost, string videoQuality, bool hasManager)
        : base(id, serviceName, cost, videoQuality) 
    {
        HasPersonalManager = hasManager;
    }
    
    public void ContactManager()
    {
        if (HasPersonalManager)
        {
            Console.WriteLine("Соединение с вашим персональным менеджером...");
        }
        else
        {
            Console.WriteLine("Эта услуга не включена в вашу подписку");
        }
    }

    // переопределяем метод еще раз
    public override string GetSubscriptionDetails()
    {
        return base.GetSubscriptionDetails() + $", Персональный менеджер: {(HasPersonalManager ? "Да" : "Нет")}";
    }
}

// шаг 2: реализация generic класса

// generic класс для управления коллекцией подписок
// ограничение where t : subscription гарантирует, что класс может работать только с подписками
public class SubscriptionManager<T> where T : Subscription
{
    private List<T> _subscriptions = new List<T>();

    // метод для добавления подписки
    public void Add(T subscription)
    {
        _subscriptions.Add(subscription);
        Console.WriteLine($"В менеджер добавлена подписка: '{subscription.ServiceName}'");
    }

    // метод для отображения всех подписок в менеджере
    public void DisplayAll()
    {
        Console.WriteLine($"\n--- Детали подписок в менеджере ({typeof(T).Name}) ---");
        if (!_subscriptions.Any())
        {
            Console.WriteLine("В менеджере нет подписок.");
            return;
        }
        
        foreach (var sub in _subscriptions)
        {
            Console.WriteLine(sub.GetSubscriptionDetails());
        }
        Console.WriteLine("-------------------------------------------");
    }
}


// создаём список для хранения различных подписок
List<Subscription> subscriptions = new List<Subscription>();

// создаем несколько экземпляров (объектов) разных классов
var onlineStorage = new OnlineServiceSubscription(101, "Облачное хранилище", 5.0, 10, "User123");
var musicService = new StreamingSubscription(202, "Музыкальный сервис", 15.0, 4, true);
var premiumVideo = new PremiumVideoSubscription(404, "Премиум Кино", 40.0, "8K HDR", true);
premiumVideo.IncludedChannels.Add("Новости Кино");
premiumVideo.IncludedChannels.Add("Классика");

subscriptions.Add(onlineStorage);
subscriptions.Add(musicService);
subscriptions.Add(premiumVideo);

Console.WriteLine("--- Демонстрация полиморфизма и наследования ---\n");

// обрабатываем все подписки в цикле
foreach (var sub in subscriptions)
{
    Console.WriteLine(sub.GetSubscriptionDetails());
    Console.WriteLine($"Итоговая ежемесячная стоимость: {sub.CalculateMonthlyCost():C}");
    Console.WriteLine($"Подписка активна уже {sub.GetSubscriptionAgeInDays()} дней");
    // здесь вызывается перекрытый метод для streamingsubscription
    sub.ExtendSubscription(6); 

    if (sub is PremiumVideoSubscription pvs)
    {
        pvs.DisplayChannels();
        pvs.ContactManager();
    }
    
    Console.WriteLine("---------------------\n");
}

Console.WriteLine("--- Демонстрация множественного наследования (интерфейсы) ---\n");
onlineStorage.UpdateProfile("Admin_Cloud");
onlineStorage.ApplyPromoCode("CLOUD20");
Console.WriteLine(onlineStorage.GetSubscriptionDetails());

Console.WriteLine("----------\n");

musicService.ToggleOfflineMode();
musicService.ApplyPromoCode("MUSIC15");
Console.WriteLine(musicService.GetSubscriptionDetails());

// демонстрация выполнения нового дополнительного задания

Console.WriteLine("\n\n--- Демонстрация перегрузки методов ---\n");
// используем разные версии метода extendsubscription
musicService.ExtendSubscription(); // вызов версии без параметров
musicService.ExtendSubscription(3); // вызов оригинальной версии (перекрытой)
musicService.ExtendSubscription(12, 10.5); // вызов версии с доп. скидкой

Console.WriteLine("\n--- Демонстрация Generic класса ---\n");
// создаем менеджер, который может хранить только streamingsubscription
var musicManager = new SubscriptionManager<StreamingSubscription>();
musicManager.Add(musicService);
// следующая строка вызовет ошибку, т.к. менеджер строго типизирован
// musicManager.Add(onlineStorage);
musicManager.DisplayAll();

// создаем менеджер, который может хранить любые типы подписок
var generalManager = new SubscriptionManager<Subscription>();
generalManager.Add(musicService);
generalManager.Add(onlineStorage);
generalManager.Add(premiumVideo);
generalManager.DisplayAll();

--- Демонстрация полиморфизма и наследования ---

ID: 101, Услуга: Облачное хранилище, Базовая стоимость: ¤5.00, Активна: True
Итоговая ежемесячная стоимость: ¤50.00
Подписка активна уже 0 дней
Подписка на 'Облачное хранилище' продлена на 6 мес.
---------------------

ID: 202, Услуга: Музыкальный сервис, Базовая стоимость: ¤15.00, Активна: True
Итоговая ежемесячная стоимость: ¤15.00
Подписка активна уже 0 дней
Подписка на 'Музыкальный сервис' продлена на 6 мес. Специальное предложение: +1 месяц в подарок!
---------------------

ID: 404, Услуга: Премиум Кино, Базовая стоимость: ¤40.00, Активна: True, Качество видео: 8K HDR, Персональный менеджер: Да
Итоговая ежемесячная стоимость: ¤40.00
Подписка активна уже 0 дней
Подписка на 'Премиум Кино' продлена на 6 мес.
Включенные каналы:
- Новости Кино
- Классика
Соединение с вашим персональным менеджером...
---------------------

--- Демонстрация множественного наследования (интерфейсы) ---

Имя пользователя для 'Облачное хранилище' изменено с 